In [85]:
import numpy as np

# Given Parameters
transmission_rate = [500, 1000, 1500, 2000, 2500, 3000]  # in kb/s
num_devices_list = [50, 100, 150, 200, 250, 300]
task_data = [5, 10, 15, 20, 25, 30, 35, 40, 45]  # in Mbit
max_delay = 0.5  # in seconds
device_computing_capacity = [0.5, 1]  # in GHz
num_of_mecs = 10
mec_computing_capacity = 4  # in GHz
transmission_power = 0.5  # in Watts

# Constants
H_m = 1e-96  # Consumption factor of electricity
alpha_t = 1  # Weighted coefficient of local time cost
alpha_e = 1  # Weighted coefficient of local energy consumption
noise_power_spectral_density = 1e-9  # in Watts/Hz
channel_gain = 1e-3  # This is a simplified assumption
path_loss_index = 4  # Typical urban area path loss exponent
bandwidth = 1e6  # 1 MHz bandwidth

#fitness function
def fitness_function(x, task_data, transmission_power, bandwidth, noise_power_spectral_density, device_computing_capacity, mec_computing_capacity):
    if x == 0:  # Local computation
        local_time_cost = task_data / device_computing_capacity
        local_energy_cost = H_m * (device_computing_capacity**2) * task_data
        total_cost = alpha_t * local_time_cost + alpha_e * local_energy_cost
    else:  # MEC computation
        transmission_time = task_data / (bandwidth * np.log2(1 + (transmission_power * channel_gain) / (noise_power_spectral_density * bandwidth)))
        mec_time_cost = task_data / mec_computing_capacity
        mec_energy_cost = transmission_power * transmission_time
        total_cost = alpha_t * (transmission_time + mec_time_cost) + alpha_e * mec_energy_cost
    return total_cost

# Adjust QPSO class to use the corrected fitness function
class QPSO:
    def __init__(self, num_devices, max_iter, task_data):
        self.num_devices = num_devices
        self.max_iter = max_iter
        self.task_data = np.array(task_data)  # Convert task data to a numpy array for easy indexing
        self.X = np.random.randint(2, size=(num_devices, max_iter))
        self.P = np.copy(self.X[:, 0:1])
        self.g = None
        self.P_fitness = np.full(num_devices, np.inf)
        self.g_fitness = np.inf

    def run(self):
        for t in range(self.max_iter):
            for i in range(self.num_devices):
                # Adjust to get a single task data size for each device
                task_size = self.task_data[i % len(self.task_data)] * 1e6  # Convert from Mbit to bit
                # Adjust to get a single device computing capacity
                device_capacity = device_computing_capacity[i % len(device_computing_capacity)] * 1e9  # Convert from GHz to Hz
                
                # Evaluate the fitness of the current position Xi
                current_fitness = fitness_function(
                    self.X[i, t], task_size, transmission_power,
                    bandwidth, noise_power_spectral_density, device_capacity, mec_computing_capacity * 1e9  # Convert from GHz to Hz
                )

                # Update personal and global bests based on fitness
                if current_fitness < self.P_fitness[i]:
                    self.P[i, 0] = self.X[i, t]
                    self.P_fitness[i] = current_fitness

                    # Update global best if necessary
                    if current_fitness < self.g_fitness:
                        self.g = np.copy(self.X[:, t])
                        self.g_fitness = current_fitness

            # Update positions for next iteration
            if t < self.max_iter - 1:
                for i in range(self.num_devices):
                    if np.random.random() < 0.5:
                        self.X[i, t+1] = self.g[i]
                    else:
                        self.X[i, t+1] = self.P[i, 0]

        return self.g
    
# Collect the best offloading strategies for each number of devices
best_offloading_strategies = {}

for num_devices in num_devices_list:
    # Initialize QPSO for the current number of devices
    qpso = QPSO(num_devices, max_iter=100, task_data=task_data)
    # Run QPSO and store the best strategy
    best_offloading_strategies[num_devices] = qpso.run()

best_offloading_strategies


{50: array([0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
        1, 1, 0, 0, 0, 0]),
 100: array([0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,
        0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1]),
 150: array([0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
        0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,
        1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 0, 0, 1

In [86]:
# Define the function to calculate mean completion time per task
def mean_completion_time(Xi, task_data, transmission_rate, device_computing_capacity, mec_computing_capacity, bandwidth, noise_power_spectral_density):
    total_time = 0
    num_tasks = len(Xi)
    
    for i, x in enumerate(Xi):
        task_size = task_data[i % len(task_data)] * 1e6  # Convert from Mbit to bit
        transmission_power = transmission_rate[i % len(transmission_rate)] * 1e3  # Convert from kb/s to b/s
        device_capacity = device_computing_capacity[i % len(device_computing_capacity)] * 1e9  # Convert from GHz to Hz
        
        if x == 0:  # Local computation
            local_time_cost = task_size / device_capacity
            total_time += local_time_cost
        else:  # MEC computation
            transmission_time = task_size / (bandwidth * np.log2(1 + (transmission_power * channel_gain) / (noise_power_spectral_density * bandwidth)))
            mec_time_cost = task_size / (mec_computing_capacity * 1e9)  # Convert from GHz to Hz
            total_time += transmission_time + mec_time_cost
    
    mean_time = total_time / num_tasks
    return mean_time

# Calculate the mean completion time per task for each number of devices
mean_completion_times = {}

for num_devices, strategy in best_offloading_strategies.items():
    mean_completion_times[num_devices] = mean_completion_time(
        strategy, task_data, transmission_rate, device_computing_capacity, mec_computing_capacity,
        bandwidth, noise_power_spectral_density)

mean_completion_times


{50: 0.5485036813706994,
 100: 0.6746988513110037,
 150: 0.6737403701885397,
 200: 0.6772979282973226,
 250: 0.6672313767985169,
 300: 0.66104281043625}

In [91]:
# Function to calculate mean completion time per each task data size
def mean_completion_time_per_task_data(task_data, transmission_rate, device_computing_capacity, mec_computing_capacity, bandwidth, noise_power_spectral_density):
    mean_times = {}
    for task_size in task_data:
        total_time_local = total_time_mec = 0
        for rate in transmission_rate:
            # Convert units for calculations
            task_size_bits = task_size * 1e6  # Convert from Mbit to bit
            transmission_power = rate * 1e3  # Convert from kb/s to b/s
            # Compute for each device computing capacity
            for capacity in device_computing_capacity:
                device_capacity_hz = capacity * 1e9  # Convert from GHz to Hz
                local_time_cost = task_size_bits / device_capacity_hz
                total_time_local += local_time_cost

                transmission_time = task_size_bits / (bandwidth * np.log2(1 + (transmission_power * channel_gain) / (noise_power_spectral_density * bandwidth)))
                mec_time_cost = task_size_bits / (mec_computing_capacity * 1e9)  # Convert from GHz to Hz
                total_time_mec += transmission_time + mec_time_cost

        # Average the times for local and MEC computations
        mean_time_local = total_time_local / (len(transmission_rate) * len(device_computing_capacity))
        mean_time_mec = total_time_mec / (len(transmission_rate) * len(device_computing_capacity))
        mean_times[task_size] = (mean_time_local + mean_time_mec) / 2  # Average of local and MEC

    return mean_times

# Calculate the mean completion time per task data
mean_completion_times_per_task_data = mean_completion_time_per_task_data(
    task_data, transmission_rate, device_computing_capacity, mec_computing_capacity,
    bandwidth, noise_power_spectral_density)

mean_completion_times_per_task_data


{5: 0.12647348287368212,
 10: 0.25294696574736425,
 15: 0.3794204486210464,
 20: 0.5058939314947285,
 25: 0.6323674143684108,
 30: 0.7588408972420928,
 35: 0.885314380115775,
 40: 1.011787862989457,
 45: 1.1382613458631394}

In [88]:
# Function to calculate system energy consumption per each task data size
def energy_consumption_per_task_data(task_data, transmission_rate, device_computing_capacity, mec_computing_capacity, H_m):
    energy_consumptions = {}
    for task_size in task_data:
        total_energy_local = total_energy_mec = 0
        for rate in transmission_rate:
            # Convert units for calculations
            task_size_bits = task_size * 1e6  # Convert from Mbit to bit
            transmission_power = rate * 1e3  # Convert from kb/s to b/s
            # Compute for each device computing capacity
            for capacity in device_computing_capacity:
                device_capacity_hz = capacity * 1e9  # Convert from GHz to Hz
                local_energy_cost = H_m * (device_capacity_hz**2) * task_size_bits
                total_energy_local += local_energy_cost

                # Assuming the same energy cost for transmission and MEC computation for simplification
                mec_energy_cost = transmission_power * task_size_bits / device_capacity_hz
                total_energy_mec += mec_energy_cost

        # Average the energy consumptions for local and MEC computations
        mean_energy_local = total_energy_local / (len(transmission_rate) * len(device_computing_capacity))
        mean_energy_mec = total_energy_mec / (len(transmission_rate) * len(device_computing_capacity))
        energy_consumptions[task_size] = (mean_energy_local + mean_energy_mec) / 2  # Average of local and MEC

    return energy_consumptions

# Calculate the system energy consumption per task data
system_energy_consumption_per_task_data = energy_consumption_per_task_data(
    task_data, transmission_rate, device_computing_capacity, mec_computing_capacity, H_m)

system_energy_consumption_per_task_data


{5: 6562.5,
 10: 13125.0,
 15: 19687.5,
 20: 26250.0,
 25: 32812.5,
 30: 39375.0,
 35: 45937.5,
 40: 52500.0,
 45: 59062.5}

In [89]:
# Function to calculate system energy consumption per transmission rate
def energy_consumption_per_transmission_rate(transmission_rate, task_data, device_computing_capacity, mec_computing_capacity, H_m):
    energy_consumptions = {}
    for rate in transmission_rate:
        total_energy_local = total_energy_mec = 0
        transmission_power = rate * 1e3  # Convert from kb/s to b/s
        for task_size in task_data:
            task_size_bits = task_size * 1e6  # Convert from Mbit to bit
            # Compute for each device computing capacity
            for capacity in device_computing_capacity:
                device_capacity_hz = capacity * 1e9  # Convert from GHz to Hz
                local_energy_cost = H_m * (device_capacity_hz**2) * task_size_bits
                total_energy_local += local_energy_cost

                # Assuming the same energy cost for transmission and MEC computation for simplification
                mec_energy_cost = transmission_power * task_size_bits / device_capacity_hz
                total_energy_mec += mec_energy_cost

        # Average the energy consumptions for local and MEC computations
        mean_energy_local = total_energy_local / (len(task_data) * len(device_computing_capacity))
        mean_energy_mec = total_energy_mec / (len(task_data) * len(device_computing_capacity))
        energy_consumptions[rate] = (mean_energy_local + mean_energy_mec) / 2  # Average of local and MEC

    return energy_consumptions

# Calculate the system energy consumption per transmission rate
system_energy_consumption_per_transmission_rate = energy_consumption_per_transmission_rate(
    transmission_rate, task_data, device_computing_capacity, mec_computing_capacity, H_m)

system_energy_consumption_per_transmission_rate


{500: 9375.0,
 1000: 18750.0,
 1500: 28125.0,
 2000: 37500.0,
 2500: 46875.0,
 3000: 56250.0}

In [90]:
# Function to calculate mean completion time per transmission rate
def completion_time_per_transmission_rate(transmission_rate, task_data, device_computing_capacity, mec_computing_capacity, bandwidth, noise_power_spectral_density):
    completion_times = {}
    for rate in transmission_rate:
        total_time_local = total_time_mec = 0
        transmission_power = rate * 1e3  # Convert from kb/s to b/s
        for task_size in task_data:
            task_size_bits = task_size * 1e6  # Convert from Mbit to bit
            # Compute for each device computing capacity
            for capacity in device_computing_capacity:
                device_capacity_hz = capacity * 1e9  # Convert from GHz to Hz
                local_time_cost = task_size_bits / device_capacity_hz
                total_time_local += local_time_cost

                transmission_time = task_size_bits / (bandwidth * np.log2(1 + (transmission_power * channel_gain) / (noise_power_spectral_density * bandwidth)))
                mec_time_cost = task_size_bits / (mec_computing_capacity * 1e9)  # Convert from GHz to Hz
                total_time_mec += transmission_time + mec_time_cost

        # Average the times for local and MEC computations
        mean_time_local = total_time_local / (len(task_data) * len(device_computing_capacity))
        mean_time_mec = total_time_mec / (len(task_data) * len(device_computing_capacity))
        completion_times[rate] = (mean_time_local + mean_time_mec) / 2  # Average of local and MEC

    return completion_times

# Calculate the mean completion time per transmission rate
mean_completion_time_per_transmission_rate = completion_time_per_transmission_rate(
    transmission_rate, task_data, device_computing_capacity, mec_computing_capacity,
    bandwidth, noise_power_spectral_density)

mean_completion_time_per_transmission_rate


{500: 0.6821477105831043,
 1000: 0.6490207789056583,
 1500: 0.6311397617345844,
 2000: 0.6190590823980083,
 2500: 0.6100135006000845,
 3000: 0.6028236519890248}